# Layered Model using `emg3d`

### Note regarding runtime

The following environment variables were set before starting Jupyter:
```
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export OPENBLAS_NUM_THREADS=1
```
This ensures that our code runs only on one thread. CPU-time is therefore the same as walltime (or even a tiny fraction smaller).

In [1]:
import emg3d
import numpy as np
import xarray as xr
from datetime import datetime
%load_ext memory_profiler

## Load model and survey

In [2]:
ds = xr.load_dataset('../block_model_and_survey.nc', engine='h5netcdf')

### Extract required info

In [3]:
# Get Mesh
hx, hy, hz = ds.attrs['hx'], ds.attrs['hy'], ds.attrs['hz']
x0 = ds.attrs['x0']
mesh_model = emg3d.utils.TensorMesh([hx, hy, hz], x0=x0)

# Get Layered Model
resh, resv = ds.attrs['resh_bg'], ds.attrs['resv_bg']

# Get Survey
src = ds.attrs['src']
strength = ds.attrs['strength']
freq = ds.attrs['freq']
rec_x = ds.x.data[::2]
rec_y = ds.attrs['rec_y']
rec_z = ds.attrs['rec_z']

# Get source center (for meshing)
src_c = np.mean(src.reshape(-1, 2), 1)

## Computation mesh

In [4]:
# Get cell widths and origin in each direction
inp = {'freq': freq, 'max_domain': 50000}
xx, x0 = emg3d.utils.get_hx_h0(res=[0.3, 1000], fixed=50, domain=[-10050, 10050], min_width=100, **inp)
yy, y0 = emg3d.utils.get_hx_h0(res=[0.3, 1000], domain=[-4000, 4000], min_width=200, **inp)
zz, z0 = emg3d.utils.get_hx_h0(res=[0.3, 100, 1000], domain=[-3150, 0], min_width=50, **inp)

# Create a TensorMesh instance.
mesh = emg3d.utils.TensorMesh([xx, yy, zz], x0=[x0, y0, z0])

   Skin depth (m/l-r)  [m] : 276 / 15916
   Survey domain       [m] : -10050 - 10050
   Calculation domain  [m] : -49950 - 50050
   Final extent        [m] : -57088 - 65201
   Min/max cell width  [m] : 100 / 100 / 8113
   Alpha survey/calc       : 1.000 / 1.170
   Number of cells (s/c/r) : 256 (201/52/3)

   Skin depth (m/l-r)  [m] : 276 / 15916
   Survey domain       [m] : -4000 - 4000
   Calculation domain  [m] : -50000 - 50000
   Final extent        [m] : -51968 - 51968
   Min/max cell width  [m] : 200 / 200 / 14624
   Alpha survey/calc       : 1.000 / 1.430
   Number of cells (s/c/r) : 64 (40/24/0)

   Skin depth (m/l/r)  [m] : 276 / 5033 / 15916
   Survey domain       [m] : -3150 - 0
   Calculation domain  [m] : -33198 - 50000
   Final extent        [m] : -35515 - 65482
   Min/max cell width  [m] : 50 / 50 / 19403
   Alpha survey/calc       : 1.000 / 1.420
   Number of cells (s/c/r) : 96 (64/32/0)



In [5]:
# Interpolate to new mesh
cresh = emg3d.utils.grid2grid(mesh_model, resh, mesh, 'volume', log=True)
cresv = emg3d.utils.grid2grid(mesh_model, resv, mesh, 'volume', log=True)

# Create model instance
model = emg3d.utils.Model(mesh, res_x=cresh, res_z=cresv)

## `emg3d` computation

In [6]:
# Source field
sfield = emg3d.utils.get_source_field(mesh, src, freq, strength=strength)

In [7]:
mem = %memit -o efield, info = emg3d.solve(mesh, model, sfield, linerelaxation=56, verb=3, return_info=True)
time = f"{info['time']:.0f} s"
ram = f"{(mem.mem_usage[0] - mem.baseline)/1024:.3f} GiB"
print(f"memory usage: {ram}")


:: emg3d START :: 14:27:38 :: v0.9.3

   MG-cycle       : 'F'                 sslsolver : False
   semicoarsening : False [0]           tol       : 1e-06
   linerelaxation : True [5 6]          maxit     : 50
   nu_{i,1,c,2}   : 0, 2, 1, 2          verb      : 3
   Original grid  : 256 x  64 x  96     => 1,572,864 cells
   Coarsest grid  :   2 x   2 x   3     => 12 cells
   Coarsest level :   7 ;   5 ;   5   

   [hh:mm:ss]  rel. error                  [abs. error, last/prev]   l s

       h_
      2h_ \                                                      /
      4h_  \                                        /\          / 
      8h_   \                            /\        /  \        /  
     16h_    \                  /\      /  \      /    \      /   
     32h_     \          /\    /  \    /    \    /      \    /    
     64h_      \    /\  /  \  /    \  /      \  /        \  /     
    128h_       \/\/  \/    \/      \/        \/          \/      

   [14:28:00]   3.753e-03  afte

In [8]:
# Extract (interpolate) Ex-field at receiver locations from the emg3d result.
egd = np.zeros((rec_x.size, 3), dtype=complex)
for i, y in enumerate(rec_y):
    egd[:, i] = emg3d.utils.get_receiver(mesh, efield.fx, (rec_x, y, rec_z))

## Save data

In [9]:
# Save the three lines
ds.line_1.data = np.vstack([egd[:, 0].real, egd[:, 0].imag]).ravel('F')
ds.line_2.data = np.vstack([egd[:, 1].real, egd[:, 1].imag]).ravel('F')
ds.line_3.data = np.vstack([egd[:, 2].real, egd[:, 2].imag]).ravel('F')

# Add info
ds.attrs['runtime'] = time
ds.attrs['n_procs'] = 1
ds.attrs['max_ram'] = ram
ds.attrs['n_cells'] = f"({mesh.nCx} x {mesh.nCy} x {mesh.nCz}) - {mesh.nC}"
ds.attrs['n_nodes'] = 'N/A'
ds.attrs['n_dof'] = 'N/A'
ds.attrs['extent'] = (f"x = {mesh.vectorNx[0]:.1f}-{mesh.vectorNx[-1]:.1f}; " # Total mesh extent
                      f"y = {mesh.vectorNy[0]:.1f}-{mesh.vectorNy[-1]:.1f}; "
                      f"z = {mesh.vectorNz[0]:.1f}-{mesh.vectorNz[-1]:.1f}")
ds.attrs['min_vol'] = f"{np.min(mesh.vol):.1f}"
ds.attrs['max_vol'] = f"{np.max(mesh.vol):.1f}"
ds.attrs['machine'] = "Laptop; i7-6600U CPU@2.6 GHz x4; 16 GB of memory, Ubuntu 18.04"
ds.attrs['version'] = f"emg3d v{emg3d.__version__}"
ds.attrs['date'] = datetime.today().isoformat()

# Save it under <{model}_{code}.nc>
ds.to_netcdf(f"../results/layered_emg3d.nc", engine='h5netcdf')

In [10]:
emg3d.Report(xr)

--------------------------------------------------------------------------------
  Date: Wed Apr 01 14:29:59 2020 CEST

                OS : Linux
            CPU(s) : 4
           Machine : x86_64
      Architecture : 64bit
               RAM : 15.5 GB
       Environment : Jupyter

  Python 3.7.6 (default, Jan  8 2020, 19:59:22)  [GCC 7.3.0]

            xarray : 0.15.0
             numpy : 1.15.4
             scipy : 1.4.1
             numba : 0.48.0
             emg3d : 0.9.3
           IPython : 7.13.0
        matplotlib : 3.1.3
--------------------------------------------------------------------------------